This notebook is used to implement every component of the system on a lecture of 2 videos.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Frame Extraction

In [ ]:
from math import floor
import cv2

frame extraction was done every minute to increase the number of testing instances

In [ ]:
%%time
for i in range(2):
  video = cv2.VideoCapture(f"/content/drive/MyDrive/Deployment_Folders/DeploymentLecture/{i+1}.mp4")
  # count the number of frames \ fps
  frames = video.get(cv2.CAP_PROP_FRAME_COUNT)
  fps = video.get(cv2.CAP_PROP_FPS)
  
  # calculate duration of the video
  seconds = floor(frames / fps)
  mins = floor(seconds/60)

  minutes = 1
  seconds = 0
  while minutes <= mins:
    frame_id = int(fps*(minutes*60 + seconds))
    video.set(cv2.CAP_PROP_POS_FRAMES, frame_id)
    ret, frame = video.read()
    #here put the path of the frames folder you created in your drive, and each time change the name of the video according to the video you have
    cv2.imwrite(r'/content/drive/MyDrive/Deployment_Folders/Deployment_Final_30_10/vid{}frame{}.png'.format(i+1,frame_id), frame)
    minutes+=1


CPU times: user 2min 57s, sys: 4.44 s, total: 3min 1s
Wall time: 4min 2s


# Detection

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone 
!pip install -r /content/yolov5/requirements.txt  # install

In [ ]:
# we can put all the frames in one folder 
!python /content/yolov5/detect.py --weights /content/drive/MyDrive/Weights/Detection_Weights.pt --img 640 --conf 0.4 --source '/content/drive/MyDrive/Deployment_Folders/Deployment_Final_30_10/*.png' --save-crop

detect: weights=['/content/drive/MyDrive/Weights/Detection_Weights.pt'], source=/content/drive/MyDrive/Deployment_Folders/Deployment_Final_30_10/*.png, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=True, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
requirements: /content/requirements.txt not found, check failed.
YOLOv5 🚀 v7.0-162-gc3e4e94 Python-3.10.11 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/98 /content/drive/MyDrive/Deployment_Folders/Deployment_Final_30_10/vid1frame10800.png: 384x640 26 faces, 41.1ms
image 2/98 /content/drive/MyDrive/Deployment_Folders/Deployment_Final_30_10/

# Recognition

In [ ]:
#importing libraries
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import os
import glob
import numpy as np
import csv

In [ ]:
# Define the model architecture
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 43)
model = model.to(device)

# Load the saved weights
saved_weights_path = "/content/Recognition_Weights_224_Final.pt"
saved_weights = torch.load(saved_weights_path)
model.load_state_dict(saved_weights)

# Switch to evaluation mode
model.eval()

#transformations 
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

#image_dir is the directory that has the cropped images resulting from the detection model
image_dir = "/content/drive/MyDrive/Deployment_Folders/Deployment_CroppedFaces"


# Get a list of all image file paths in the directory
image_paths = glob.glob(os.path.join(image_dir, "*.jpg"))

# Create a list to store the predicted labels
predicted_labels = []

for path in image_paths:
    #Open the image
    image = Image.open(path)
    
    #transformations
    image = transform(image).unsqueeze(0)
    
    #make predictions
    with torch.no_grad():
        image = image.to(device)
        output = model(image)
        _, predicted = torch.max(output.data, 1)
        predicted_labels.append(predicted.item())

#Print the predicted labels
num_to_name = {'abdelnasser abuziuter': 0, 'abdelrahman elayyan': 1, 'ahmad ghuneim': 2, 'aya ladadweh': 3, 'aya sulaq': 4, 
               'ban qaqish': 5, 'bana hjeji': 6, 'batool barakat': 7, 'christine amareen': 8, 'dana twal ': 9, 'fahd othman': 10, 
               'hashem thabsem': 11, 'jana shaer': 12, 'kholoud qubbaj': 13, 'lara diab': 14, 'malak abdelwahab': 15, 'mohammad jumaa': 16, 
               'noor alawi': 17, 'noor awwad': 18, 'osama zamel': 19, 'raghad abu tarboush': 20, 'rakan armoush': 21, 'rama al alawneh': 22, 
               'raneem nabulsea': 23, 'reem assi': 24, 'rose al nairab': 25, 'saif aburaisi': 26, 'saja taweel': 27, 'samira abubakr': 28, 
               'sanad abu khalaf': 29, 'sara darwish': 30, 'sara selwadi': 31, 'shahem al naber': 32, 'suhaib abu kiwan': 33, 'suheil wakileh': 34, 
               'tamara kawamleh': 35, 'tareq sarayji': 36, 'tariq sallam': 37, 'tasneem sanuri': 38, 'waleed abujaish': 39, 'yara matarneh': 40, 
               'yousef shaqadan': 41, 'zain shaarawi': 42}
inverted_dict = {v: k for k, v in num_to_name.items()}
students = num_to_name.keys()

result = []
for i in predicted_labels:
    if predicted_labels.count(i) >= 1 and i not in result:
        result.append(i)
result_names = [inverted_dict[num] for num in result]
print(result_names)

attendance_dict = {student: 1 if student in result_names else 0 for student in students}

# write attendance to CSV file
with open('attendance.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Name', 'Attendance'])
    for student in students:
        writer.writerow([student, attendance_dict[student]])